<a href="https://colab.research.google.com/github/pokjay/heb-squad/blob/main/preprocess_translate_squad_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import pandas as pd

## Convert SQuAD JSON file to Pandas DataFrame

In [2]:
!git clone https://github.com/pokjay/heb-squad

Cloning into 'heb-squad'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 47 (delta 11), reused 33 (delta 5), pack-reused 0
Unpacking objects: 100% (47/47), done.
Checking out files: 100% (17/17), done.


In [3]:
with open('/content/heb-squad/data/original/dev-v2.0.json') as f_handle:
    dataset = json.load(f_handle)

In [4]:
def handle_answer(article, context, qa, answer_field):

  rows = []

  for answer in qa[answer_field]:

    q = qa['question']
    qid = qa['id']
    is_impossible = qa['is_impossible']
    answer_end = answer['answer_start'] + len(answer['text'])
                
    context_marked = context[:answer['answer_start']] + \
                    '[0123] ' + \
                    context[answer['answer_start'] : answer_end] + \
                    ' [4567]' \
                    + context[answer_end:]
                
    row = {'article': article['title'],
           'id': qid,
           'context': context_marked,
           'question': q,
           'answer': answer['text'],
           'answer_start': answer['answer_start'],
           'answer_end': answer_end,
           'is_impossible': int(is_impossible)}

    rows.append(row)
  
  return rows

In [5]:
rows = []

for article in dataset['data']:
  for p in article['paragraphs']:
    # Some lines have \n, we remove them as they screw up translation
    context = p['context'].replace('\n', '')
    
    for qa in p['qas']:
      if 'answers' in qa:
        rows.extend(handle_answer(article, context, qa, 'answers'))
      if 'plausible_answers' in qa:
        rows.extend(handle_answer(article, context, qa, 'plausible_answers'))
      


In [6]:
df = pd.DataFrame(rows).fillna("")
df.index = df.index.rename('idx')
df.shape

(26232, 8)

In [7]:
df.head()

,article,id,context,question,answer,answer_start,answer_end,is_impossible
idx,,,,,,,,
0,Normans,56ddde6b9a695914005b9628,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,France,159,165,0
1,Normans,56ddde6b9a695914005b9628,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,France,159,165,0
2,Normans,56ddde6b9a695914005b9628,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,France,159,165,0
3,Normans,56ddde6b9a695914005b9628,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,France,159,165,0
4,Normans,56ddde6b9a695914005b9629,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,10th and 11th centuries,94,117,0


In [8]:
# df.drop(columns=['article']).to_excel('/content/heb-squad/data/intermediate/dev-v2.0.xlsx', index=True)

## 1. Translate XLSX manually using Google Translate Document translator
## 2. Run code below processing the translated output

In [9]:
df2 = pd.read_csv('/content/heb-squad/data/intermediate/heb-dev-v2.0.tsv', sep='\t', quoting=3, index_col=0)

In [10]:
df2['id'] = df2.id.str.strip()
df2['context'] = df2.context.str.strip()
df2['question'] = df2.question.str.strip()
df2['answer'] = df2.answer.str.strip()

Take a quick look at what we got

In [11]:
df2.shape

(26232, 7)

In [12]:
df2.head(3)

,id,context,question,answer,answer_start,answer_end,is_impossible
idx,,,,,,,
0,56ddde6b9a695914005b9628,הנורמנים (נורמן: נורמנדים; צרפתית: נורמנדים; ל...,באיזו מדינה נמצאת נורמנדי?,צָרְפַת,159,165,0
1,56ddde6b9a695914005b9628,הנורמנים (נורמן: נורמנדים; צרפתית: נורמנדים; ל...,באיזו מדינה נמצאת נורמנדי?,צָרְפַת,159,165,0
2,56ddde6b9a695914005b9628,הנורמנים (נורמן: נורמנדים; צרפתית: נורמנדים; ל...,באיזו מדינה נמצאת נורמנדי?,צָרְפַת,159,165,0


### Validate IDs are correct, fix if needed

Check how many diffs we have

In [13]:
pd.Series(df.id.values == df2.id.values).value_counts()

True     26152
False       80
dtype: int64

Print problematic IDs

In [14]:
df2.loc[~(df.id.values == df2.id.values), 'id']

idx
34                                 28
528       תשלום 56e19557e3433e1400422
529       תשלום 56e19557e3433e1400422
530       תשלום 56e19557e3433e1400422
915          56e1cbe2cd28a01900c67 רע
                     ...             
25405           5737534ec3c5551400e51
25406           5737534ec3c5551400e51
25407           5737534ec3c5551400e51
25408           5737534ec3c5551400e51
25409           5737534ec3c5551400e51
Name: id, Length: 80, dtype: object

Print original IDs

In [15]:
df.loc[~(df.id.values == df2.id.values), 'id']

idx
34       5ad3a266604f3c001a3fea28
528      56e19557e3433e1400422fee
529      56e19557e3433e1400422fee
530      56e19557e3433e1400422fee
915      56e1cbe2cd28a01900c67bad
                   ...           
25405    5737534ec3c5551400e51ead
25406    5737534ec3c5551400e51ead
25407    5737534ec3c5551400e51ead
25408    5737534ec3c5551400e51ead
25409    5737534ec3c5551400e51ead
Name: id, Length: 80, dtype: object

Replace problematic ids with original ids

In [16]:
df2.loc[~(df.id.values == df2.id.values), 'id'] = df.loc[~(df.id.values == df2.id.values), 'id'].to_list()

In [17]:
pd.Series(df.id.values == df2.id.values).value_counts()

True    26232
dtype: int64

### Add name of article for creating JSON later

In [18]:
df2['article'] = df.article.to_list()
df2.article.value_counts() == df.article.value_counts()

Oxygen                                       True
Rhine                                        True
Economic_inequality                          True
Civil_disobedience                           True
Force                                        True
Imperialism                                  True
French_and_Indian_War                        True
European_Union_law                           True
Warsaw                                       True
Steam_engine                                 True
Immune_system                                True
Yuan_dynasty                                 True
Huguenot                                     True
Prime_number                                 True
Computational_complexity_theory              True
Islamism                                     True
University_of_Chicago                        True
Scottish_Parliament                          True
Southern_California                          True
Ctenophora                                   True


Remove markers from context, and then print a sample to see if context looks good

In [19]:
df2['context_marked'] = df2.context

In [20]:
df2['context'] = df2.context_marked.str.replace('[0123] ', '', regex=False)
df2['context'] = df2.context.str.replace(' [4567]', '', regex=False)

In [21]:
df2.iloc[10].context_marked, '', '', df2.iloc[10].context

('הנורמנים (נורמן: נורמנדים; בצרפתית: נורמנדים; בלטינית: Normanni) היו האנשים שבמאות ה -10 וה -11 נתנו את שמם לנורמנדי, אזור בצרפת. הם צאצאים מפושעים ופיראטים נורדים ("נורמן" מגיע מ"נורסמן ") ודנאטים מ [0123] דנמרק, איסלנד ונורווגיה [4567], שתחת מנהיגם רולו הסכימו להישבע לנאמנות למלך צ\'ארלס השלישי ממערב פרנסיה. באמצעות דורות של התבוללות והתערבבות עם האוכלוסייה הפרנקאית והרומית-גאלית הילידית, צאצאיהם היו מתמזגים בהדרגה עם התרבויות הקרולינגיות של מערב פרנסיה המערבית. הזהות התרבותית והאתנית המובהקת של הנורמנים התפתחה בתחילה במחצית הראשונה של המאה ה -10, והיא המשיכה להתפתח במהלך המאות הבאות.',
 '',
 '',
 'הנורמנים (נורמן: נורמנדים; בצרפתית: נורמנדים; בלטינית: Normanni) היו האנשים שבמאות ה -10 וה -11 נתנו את שמם לנורמנדי, אזור בצרפת. הם צאצאים מפושעים ופיראטים נורדים ("נורמן" מגיע מ"נורסמן ") ודנאטים מ דנמרק, איסלנד ונורווגיה, שתחת מנהיגם רולו הסכימו להישבע לנאמנות למלך צ\'ארלס השלישי ממערב פרנסיה. באמצעות דורות של התבוללות והתערבבות עם האוכלוסייה הפרנקאית והרומית-גאלית הילידית, צאצאיהם הי

Calculate the new start and end locations of the answer in the Hebrew context

In [22]:
df2['answer_start_heb'] = df2.context_marked.str.find('[0123] ')
df2['answer_end_heb'] = df2.apply(lambda row: row.context_marked.find(' [4567]', int(row.answer_start_heb)) - len(' [4567]'), axis=1)

Validate how many rows are problematic and will be discarded

In [23]:
print((df2.answer_start_heb >= 0).value_counts())
print()
print((df2.answer_end_heb >= 0).value_counts())

True     22566
False     3666
Name: answer_start_heb, dtype: int64

True     22167
False     4065
Name: answer_end_heb, dtype: int64


In [24]:
for row in df2.sample(30):
  df2.iloc[10].context[int(df2.iloc[10].answer_start_heb):int(df2.iloc[10].answer_end_heb)]

In [25]:
for row in df2.sample(10).itertuples():
  print(row.question)
  print(row.context[int(row.answer_start_heb):int(row.answer_end_heb)])
  print()

פרט לחוקי הקרקעות, ממה עוד לא היו הקליפורנים לא מרוצים?
מסים לא שוויוניים

מהן התרופות האנטי דלקתיות הפחות חזקות?
גלוקוקורטיקואידים

מה היה היחס בין המתנחל הבריטי לצרפתי?
20 עד 1

מהי המדידה המקובלת של הריין?
קילומטר הריין

מהו החוק העיקרי של האיחוד האירופי?
מורכב בעיקר מהסכמי היסוד

מהי סוגיה אחת המוסיפה את מורכבות עבודת הרוקח?
שיטות הטיפול

מה חשבו האירופאים שהאנשים באזורים הטרופיים זקוקים להם?


מי היה יו"ר ה- IPCC הראשון?
ברט בולין

מי היה המנהיג הנורדי?
רולו

באיזה מסמך הודו ההוגנוטים באמונתם בפני הפורטוגלים בברזיל?




#### Try and fix answers ruined in translation
These are rows where the tags surrounding the answer was lost in translation, maeaning we cannot retrieve the correct indices of the answers from the Hebrew translation. We will try to use the translated answer to find the indices in the context.

In [26]:
mis_ans_df = df2[(df2.answer_start_heb >= df2.answer_end_heb) | (df2.answer_start_heb < 0) | (df2.answer_end_heb < 0)]

In [27]:
mis_ans_df.shape

(5302, 11)

In [28]:
new_ans_start = mis_ans_df.apply(lambda row: row.context.find(row.answer), axis=1)
new_ans_end = mis_ans_df.apply(lambda row: -1 if (row.context.find(row.answer) == -1) else (row.context.find(row.answer) + len(row.answer)), axis=1)

new_ans_start_success = new_ans_start[new_ans_start > -1]
new_ans_end_success = new_ans_end[new_ans_end > -1]

df2.loc[new_ans_start_success.index, 'answer_start_heb'] = new_ans_start_success.to_list()
df2.loc[new_ans_end.index, 'answer_end_heb'] = new_ans_end.to_list()

In [29]:
print((df2.answer_start_heb >= 0).value_counts())
print()
print((df2.answer_end_heb >= 0).value_counts())

True     24273
False     1959
Name: answer_start_heb, dtype: int64

True     23307
False     2925
Name: answer_end_heb, dtype: int64


Below we remove rows where the translation deleted the answer markers from the context.

In [30]:
print('We will remove', df2[(df2.answer_start_heb >= df2.answer_end_heb) | (df2.answer_start_heb < 0) | (df2.answer_end_heb < 0)].shape[0], 'rows')

We will remove 2925 rows


In [31]:
df2_clean = df2[(df2.answer_start_heb < df2.answer_end_heb) | (df2.answer_start_heb >= 0) | (df2.answer_end_heb >= 0)]

In [32]:
df2_clean.shape

(24273, 11)

Now we can save the df to file, it is ready!

In [33]:
!mkdir /content/heb-squad/data/final

In [34]:
df2_clean.to_csv('/content/heb-squad/data/final/heb-dev-v2.0.csv', index=False)